In [1]:
!sudo pip install -q transformers --upgrade

In [ ]:
!sudo pip install -q autoawq

In [ ]:
import os
import torch
from datasets import load_dataset
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd
import torch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
    model = AutoModelForCausalLM.from_pretrained("my_mistral_model")
    tokenizer = AutoTokenizer.from_pretrained("my_mistral_model")